# CFAR10

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# Load the CIFAR10 dataset
cifar10 = tf.keras.datasets.cifar10.load_data()

(X_train_full, y_train_full), (X_test, y_test) = cifar10

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [4]:
X_train.shape

(45000, 32, 32, 3)

In [5]:
len(np.unique(y_train))

10

##### Build a DNN :

* 20 hidden layers , 100 neurons each
* Swish Activation
* He Initialization


In [6]:
tf.random.set_seed(42)

SwishHeDense = partial(
    tf.keras.layers.Dense,
    activation = "swish",
    kernel_initializer="he_normal"
)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(SwishHeDense(100))

* Nadam Optimization
* Early Stopping
* Softmax Output layer

In [7]:
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [8]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(
    loss = "sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics = ['accuracy']
)

In [9]:
from pathlib import Path

In [10]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("cifar10_model",
                                                         save_best_only=True)

run_index = 1 # increment every time you train the model
run_logdir = Path() / "cifar10_logs" / f"run_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [12]:
#%load_ext tensorboard
#%tensorboard --logdir=./cifar10_logs

In [13]:
model.fit(X_train, y_train, 
          epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 69s 41ms/step - loss: 4.8990 - accuracy: 0.1556 - val_loss: 2.1209 - val_accuracy: 0.2126
Epoch 2/100
1407/1407 [==============================] - 54s 38ms/step - loss: 2.0520 - accuracy: 0.2396 - val_loss: 2.0103 - val_accuracy: 0.2500
Epoch 3/100
1407/1407 [==============================] - 53s 37ms/step - loss: 1.9452 - accuracy: 0.2834 - val_loss: 1.9279 - val_accuracy: 0.2864
Epoch 4/100
1407/1407 [==============================] - 54s 39ms/step - loss: 1.8756 - accuracy: 0.3107 - val_loss: 1.9115 - val_accuracy: 0.3090
Epoch 5/100
1407/1407 [==============================] - 55s 39ms/step - loss: 1.8160 - accuracy: 0.3368 - val_loss: 1.8083 - val_accuracy: 0.3340
Epoch 6/100
1407/1407 [==============================] - 57s 40ms/step - loss: 1.7744 - accuracy: 0.3541 - val_loss: 1.7628 - val_accuracy: 0.3588
Epoch 7/100
1407/1407 [==============================] - 55s 39ms/step - loss: 1.7314 - accuracy: 0.3714 - val_loss: 1

In [14]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 3s 18ms/step - loss: 1.5098 - accuracy: 0.4588


[1.5098285675048828, 0.45879998803138733]

* Add Batch Normalization